# Generating Weekday and Weekend profiles

In [39]:
import polars as pl
import glob
import re
import os

## Weekday profiles

In [40]:
#load spring files first
# Function to extract numeric part from file name
season= "spring"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None

# Get the list of file paths
file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv') #location of spring season files
file_paths = sorted(file_paths, key=extract_numeric_part)
print(file_paths[:5]) #check

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\0_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10000_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10001_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10002_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10003_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10004_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10005_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10006_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10007_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10008_spring_data.csv']


In [41]:
## add the holiday dates to exclude
def weekday_extraction(file_path):
    """ This function extracts weekday data from the seasonal files.
         Update the holidays_to_exclude to remove the holidays from weekdays"""
    # Load the data into a Polars DataFrame
    data = pl.read_csv(file_path)

    # Ensure the timestamp column is treated as a string
    data = data.with_columns(pl.col('timestamp').cast(pl.Utf8))
    #print(data, "1")
    # Parse the timestamp column to datetime with the correct format and strict=False
    data = data.with_columns(pl.col('timestamp').str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%.fZ", strict=False))
    #print(data, "2")
    # Extract the day of the week and the time of day from the timestamp
    data = data.with_columns([
        pl.col('timestamp').dt.strftime('%Y-%m-%d').alias('date'),
        pl.col('timestamp').dt.weekday().alias('day_of_week'),
        pl.col('timestamp').dt.strftime('%H:%M:%S').alias('time_of_day'),
        pl.lit('spring').alias('season')
    ])
    #print(data, "3")

    # Filter out weekends (Monday=1, ..., Sunday=7)
    weekdays = [1, 2, 3, 4,5]
    data = data.filter(pl.col('day_of_week').is_in(weekdays))

    # Define holidays to exclude
    holidays_to_exclude = ['2021-06-03', '2021-04-05', '2021-05-13']
    #holidays_to_exclude = ['2022-01-01']
    # Filter out holidays
    data = data.filter(~pl.col('date').is_in(holidays_to_exclude))

    # Group by day of the week and time of day
    grouped = data.group_by(['day_of_week', 'time_of_day'])

    # Calculate the mean for each group
    daily_averages = grouped.agg(pl.col('value_kwh').mean().alias('average_kwh'))
    #print(data,"3")
    # Pivot the data to have days of the week as columns and time of day as rows
    pivot_table = daily_averages.pivot(
        values='average_kwh',
        index='time_of_day',
        columns='day_of_week'
    ).sort('time_of_day')

    # Replace the day_of_week index with actual day names
    day_mapping = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday'}
    pivot_table = pivot_table.rename({str(day): name for day, name in day_mapping.items()})
    #pivot_table= pivot_table.with_columns(pl.lit(f'{season}').alias('season')) #if you want a column with season name
    #print(f'{season} table',pivot_table)
    # Show the resulting pivot table
    return(pivot_table)

In [42]:
for idx,file_path in enumerate(file_paths):
   
    weekdays_df = weekday_extraction(file_path)

    # saving the files. update path
    weekdays_df.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekdays\{idx}_weekdays_data.csv')

    if idx%1000 ==0:
        print(f"{season} weekday files generated {idx} of {len(file_paths)}")

spring weekday files generated 0 of 10


C:\Users\pana\AppData\Local\Temp\ipykernel_13980\3692816435.py:40: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivot_table = daily_averages.pivot(


In [43]:
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_weekdays_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None

# Get the list of file paths
file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekdays\*_weekdays_data.csv') #same path as before
file_paths = sorted(file_paths, key=extract_numeric_part)
print(file_paths[:10])

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\0_weekdays_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\1_weekdays_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\2_weekdays_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\3_weekdays_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\4_weekdays_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\5_weekdays_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\6_weekdays_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\7_weekdays_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\8_weekdays_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekdays\\9_weekdays_data.csv']


In [44]:
season= "summer"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None
    
#path were summer season files are
toadd_file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv') 
toadd_file_paths = sorted(toadd_file_paths, key=extract_numeric_part)
print(toadd_file_paths[:5])

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\0_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10000_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10001_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10002_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10003_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10004_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10005_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10006_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10007_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10008_summer_data.csv']


In [45]:
def append_season_weekday_averages(existing_file_path, summer_file_path):
    # Load existing data into a Polars DataFrame
    existing_data = pl.read_csv(existing_file_path)
    #print("existing data", existing_data)

    # Load summer data
    table_summer = weekday_extraction(summer_file_path)
    
    # Ensure columns match between existing_data and pivot_table_summer
    existing_data = existing_data.select(table_summer.columns)

    # Concatenate existing data with summer pivot_table
    concatenated_data = pl.concat([existing_data, table_summer])

    # Return the concatenated DataFrame
    return concatenated_data

for idx,file_path in enumerate(file_paths):
   
    existing_file = file_path
    summer_file = toadd_file_paths[i]
    #print(summer_file,existing_file)
    result = append_season_weekday_averages(existing_file, summer_file)

    # Print or save the result
    result.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekdays\{idx}_weekdays_data.csv')

    if idx%1000 ==0:
        print(f"{season} weekday files add to existing weekday file {idx} of {len(file_paths)}")

summer weekday files add to existing weekday file 0 of 10


C:\Users\pana\AppData\Local\Temp\ipykernel_13980\3692816435.py:40: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivot_table = daily_averages.pivot(


In [46]:
season= "autumn"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None
    
#path were summer season files are
toadd_file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv') 
toadd_file_paths = sorted(toadd_file_paths, key=extract_numeric_part)
print(toadd_file_paths[:5])

for idx,file_path in enumerate(file_paths):
   
    existing_file = file_path
    summer_file = toadd_file_paths[i]
    
    result = append_season_weekday_averages(existing_file, summer_file)

    # Print or save the result
    result.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekdays\{idx}_weekdays_data.csv')

    if idx%1000 ==0:
        print(f"{season} weekday files add to existing weekday file {idx} of {len(file_paths)}")

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\0_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10000_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10001_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10002_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10003_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10004_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10005_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10006_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10007_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10008_autumn_data.csv']
autumn weekday files add to existing weekday file 0 of 10


C:\Users\pana\AppData\Local\Temp\ipykernel_13980\3692816435.py:40: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivot_table = daily_averages.pivot(


In [47]:
season= "winter"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None
    
#path were summer season files are
toadd_file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv') 
toadd_file_paths = sorted(toadd_file_paths, key=extract_numeric_part)
print(f" the {season} files are:",toadd_file_paths[:5])

for idx,file_path in enumerate(file_paths):
   
    existing_file = file_path
    summer_file = toadd_file_paths[i]
    
    result = append_season_weekday_averages(existing_file, summer_file)

    # Print or save the result
    result.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekdays\{idx}_weekdays_data.csv')

    if idx%1000 ==0:
        print(f"{season} weekday files add to existing weekday file {idx} of {len(file_paths)}")

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\0_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10000_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10001_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10002_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10003_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10004_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10005_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10006_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10007_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10008_winter_data.csv']
winter weekday files add to existing weekday file 0 of 10


C:\Users\pana\AppData\Local\Temp\ipykernel_13980\3692816435.py:40: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivot_table = daily_averages.pivot(


In [48]:
standard_order = ['time_of_day', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

def weekdays_avg(df):
    
    #df = df[standard_order]
    df = df.with_columns(
        weekdays_avg = pl.mean_horizontal("Monday","Tuesday","Wednesday","Thursday","Friday")
        )
    max_df= df.select(pl.max("weekdays_avg"))
    max_df= max_df['weekdays_avg'][0]
    #print(max_df)
    if max_df ==0:
        df = df.with_columns(
            normalized_weekdays = pl.col("weekdays_avg")
        )
    else:
        df = df.with_columns(
            normalized_weekdays = pl.col("weekdays_avg")/max_df
        )

    df = df.sort("time_of_day")
    return df

In [49]:
# Function to process each file
def process_file(df):
    """ This function groups by the time of day for each day of the week"""
    # Group by the 'time' column and calculate the mean for each group
    grouped_df = df.group_by("time_of_day").agg([
        pl.mean("Monday").alias("Monday"),
        pl.mean("Tuesday").alias("Tuesday"),
        pl.mean("Wednesday").alias("Wednesday"),
        pl.mean("Thursday").alias("Thursday"),
        pl.mean("Friday").alias("Friday")
    ])

    return grouped_df

#saves the average week for each household in the 'week' folder
for idx,file_path in enumerate(file_paths):
    df = pl.read_csv(file_path)
    df = df[standard_order]
    #df = df.with_columns(pl.col('time_of_day').str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%.fZ", strict=False))
    
    df_reduced = process_file(df)
    
    df_normalized = weekdays_avg(df_reduced)
    
    df_normalized.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekdays\{idx}_weekdays_data.csv')
    if idx%1000 == 0:
        print(f"Processed {idx}/{len(file_paths)}")

#Average week profiles for the whole year is processed

Processed 0/10


# Weekend Profiles

In [50]:
#load spring files first
# Function to extract numeric part from file name
season= "spring"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None

# Get the list of file paths
file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv') #location of spring season files
file_paths = sorted(file_paths, key=extract_numeric_part)
print(f"{seasonal} files location:",file_paths[:5])

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\0_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10000_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10001_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10002_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10003_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10004_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10005_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10006_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10007_spring_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\spring\\10008_spring_data.csv']


In [51]:
def weekend_extraction(file_path):
    # Load the data into a Polars DataFrame
    data = pl.read_csv(file_path)

    # Ensure the timestamp column is treated as a string
    data = data.with_columns(pl.col('timestamp').cast(pl.Utf8))

    # Parse the timestamp column to datetime with the correct format and strict=False
    data = data.with_columns(pl.col('timestamp').str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%.fZ", strict=False))

    # Extract the day of the week and the time of day from the timestamp
    data = data.with_columns([
        pl.col('timestamp').dt.strftime('%Y-%m-%d').alias('date'),
        pl.col('timestamp').dt.weekday().alias('day_of_week'),
        pl.col('timestamp').dt.strftime('%H:%M:%S').alias('time_of_day'),
        pl.lit('spring').alias('season')
    ])
    #print(data, "2")

    # Filter out weekends (Monday=1, ..., Sunday=7)
    weekend = [6,7]
    data = data.filter(pl.col('day_of_week').is_in(weekend))

    # Group by day of the week and time of day
    grouped = data.group_by(['day_of_week', 'time_of_day'])

    # Calculate the mean for each group
    daily_averages = grouped.agg(pl.col('value_kwh').mean().alias('average_kwh'))

    # Pivot the data to have days of the week as columns and time of day as rows
    pivot_table = daily_averages.pivot(
        values='average_kwh',
        index='time_of_day',
        columns='day_of_week'
    ).sort('time_of_day')

    # Replace the day_of_week index with actual day names
    day_mapping = {6: 'Saturday', 7: 'Sunday'}
    pivot_table = pivot_table.rename({str(day): name for day, name in day_mapping.items()})
    #pivot_table= pivot_table.with_columns(pl.lit(f'{season}').alias('season')) #if you want a column with season name,

    #print(f'{season} table',pivot_table)
    
    return(pivot_table)

In [52]:
for idx,file_path in enumerate(file_paths):
   
    weekend_df = weekend_extraction(file_path)

    # saving the files. update path
    weekend_df.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekend\{idx}_weekend_data.csv')

    if idx%1000 ==0:
        print(f"{season} weekday files generated {idx} of {len(file_paths)}")

spring weekday files generated 0 of 10


C:\Users\pana\AppData\Local\Temp\ipykernel_13980\2303263902.py:31: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivot_table = daily_averages.pivot(


In [53]:
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_weekend_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None

# Get the list of file paths
file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekend\*_weekend_data.csv') #same path as previous cell
file_paths = sorted(file_paths, key=extract_numeric_part)
print(file_paths[:10])

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\0_weekend_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\1_weekend_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\2_weekend_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\3_weekend_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\4_weekend_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\5_weekend_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\6_weekend_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\7_weekend_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\8_weekend_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2022\\weekend\\9_weekend_data.csv']


In [54]:
standard_order = ['time_of_day','Saturday','Sunday'] #add 'season' here if you have a column with season names
def weekend_avg(data):
    df= data
    df = df[standard_order]
    df = df.with_columns(
        weekend_avg = pl.mean_horizontal("Saturday","Sunday")
        )
    max_df= df.select(pl.max("weekend_avg"))
    max_df= max_df['weekend_avg'][0]
    #print(max_df)
    if max_df ==0:
        df = df.with_columns(
            normalized_weekend = pl.col("weekend_avg")
        )
    else:
        df = df.with_columns(
            normalized_weekend = pl.col("weekend_avg")/max_df
        )
    return df

In [55]:
season= "summer"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None
    
#path were summer season files are
toadd_file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv') 
toadd_file_paths = sorted(toadd_file_paths, key=extract_numeric_part)
print(toadd_file_paths[:5])

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\0_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10000_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10001_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10002_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10003_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10004_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10005_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10006_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10007_summer_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\summer\\10008_summer_data.csv']


In [56]:
def append_season_weekend_averages(existing_file_path, summer_file_path):
    # Load existing data into a Polars DataFrame
    existing_data = pl.read_csv(existing_file_path)
    existing_data = existing_data[standard_order]
    #print("existing data", existing_data)

    # Load summer data
    table_summer = weekend_extraction(summer_file_path)
    #print("table_summer",table_summer)
    table_summer = table_summer[standard_order]
    
    # Ensure columns match between existing_data and pivot_table_summer
    existing_data = existing_data.select(table_summer.columns)

    # Concatenate existing data with summer pivot_table
    concatenated_data = pl.concat([existing_data, table_summer])

    # Return the concatenated DataFrame
    return concatenated_data

# Example usage:
for idx,file_path in enumerate(file_paths):
    existing_file = file_path
    summer_file = toadd_file_paths[i]
    #print(summer_file,existing_file)
    result = append_season_weekend_averages(existing_file, summer_file)

    # Print or save the result
    result.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekend\{idx}_weekend_data.csv')
    if idx%1000 ==0:
        print(f"{season} weekday files generated {idx} of {len(file_paths)}")

summer weekday files generated 0 of 10


C:\Users\pana\AppData\Local\Temp\ipykernel_13980\2303263902.py:31: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivot_table = daily_averages.pivot(


In [57]:
season= "autumn"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None
    
#path were summer season files are
toadd_file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv') 
toadd_file_paths = sorted(toadd_file_paths, key=extract_numeric_part)
print(toadd_file_paths[:5])

for idx,file_path in enumerate(file_paths):
   
    existing_file = file_path
    summer_file = toadd_file_paths[i]
    
    result = append_season_weekend_averages(existing_file, summer_file)

    # Print or save the result
    result.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekend\{idx}_weekend_data.csv')

    if idx%1000 ==0:
        print(f"{season} weekday files add to existing weekday file {idx} of {len(file_paths)}")

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\0_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10000_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10001_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10002_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10003_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10004_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10005_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10006_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10007_autumn_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\autumn\\10008_autumn_data.csv']
autumn weekday files add to existing weekday file 0 of 10


C:\Users\pana\AppData\Local\Temp\ipykernel_13980\2303263902.py:31: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivot_table = daily_averages.pivot(


In [58]:
season= "winter"
def extract_numeric_part(file_path):
    match = re.search(rf'(\d+)_{season}_data\.csv', file_path)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Filename {file_path} does not match pattern")
        return None
    
#path were summer season files are
toadd_file_paths = glob.glob(rf'C:\Users\pana\Desktop\DATA\ckw\2021\{season}\*_{season}_data.csv') 
toadd_file_paths = sorted(toadd_file_paths, key=extract_numeric_part)
print(toadd_file_paths[:5])

for idx,file_path in enumerate(file_paths):
   
    existing_file = file_path
    summer_file = toadd_file_paths[i]
    
    result = append_season_weekend_averages(existing_file, summer_file)

    # Print or save the result
    result.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekend\{idx}_weekend_data.csv')

    if idx%1000 ==0:
        print(f"{season} weekend files added to existing weekend file {idx} of {len(file_paths)}")

['C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\0_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10000_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10001_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10002_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10003_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10004_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10005_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10006_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10007_winter_data.csv', 'C:\\Users\\pana\\Desktop\\DATA\\ckw\\2021\\winter\\10008_winter_data.csv']
winter weekend files added to existing weekend file 0 of 10


C:\Users\pana\AppData\Local\Temp\ipykernel_13980\2303263902.py:31: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  pivot_table = daily_averages.pivot(


In [59]:
# Function to process each file
def process_file(df):
    """ This function groups by the time of day for each day of the week"""
    # Group by the 'time' column and calculate the mean for each group
    grouped_df = df.group_by("time_of_day").agg([
        pl.mean("Saturday").alias("Saturday"),
        pl.mean("Sunday").alias("Sunday")
    ])

    return grouped_df

#saves the average week for each household in the 'week' folder
for idx,file_path in enumerate(file_paths):
    df = pl.read_csv(file_path)
    df = df[standard_order]
    #df = df.with_columns(pl.col('time_of_day').str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%.fZ", strict=False))
    
    df_reduced = process_file(df)
    
    df_normalized = weekend_avg(df_reduced)
    
    df_normalized.write_csv(rf'C:\Users\pana\Desktop\DATA\ckw\2022\weekend\{idx}_weekend_data.csv')
    if idx%1000 == 0:
        print(f"Processed {idx}/{len(file_paths)}")

print("weekend files generated")
#Average week profiles for the whole year is processed

Processed 0/10
